In [ ]:
import os
from pathlib import Path
notebook_dir = Path.cwd()
layout_dir = notebook_dir.parent.parent / 'layout'
os.chdir(layout_dir)
import gdsfactory as gf
from blocks import *
from comb_drive_tuning import *
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
c = spring_5um(spring_length=50, spring_width=2,separation=15, num_loops=2)
c_wg = c.extract(['WG'])

c.write_gds(notebook_dir / 'spring_5um.gds')

In [ ]:
import mph
client = mph.start()

In [ ]:
model = client.load(notebook_dir/'spring.mph')
jmodel = model.java

In [ ]:
imp1 = model/"geometries"/"Geometry 1"/"Import 1"

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import time
def simulate_stiffness(length, width, loop_num):
    row = dict(
        spring_length=float(length),
        spring_width=float(width),
        num_loops=int(loop_num),
        separation=float(15),
        gds_file="",
        xsize=np.nan,
        kxx=np.nan, kyy=np.nan, kzz=np.nan,
        status="fail",
        error=""
    )

    try:
        # --- 1) build parametric layout ---
        c = spring_5um(
            spring_length=length,
            spring_width=width,
            separation=15,
            num_loops=loop_num
        )
        c_wg = c.extract(['WG'])
        xsize = float(c_wg.xsize)
        row["xsize"] = xsize

        # --- 2) write per-case gds + import into COMSOL ---
        # Use a deterministic filename; keep some decimals safe for filenames.
        w_tag = f"{width}".replace(".", "p")
        gds_path = out_dir / f"spring_L{length}_W{w_tag}_N{loop_num}_S{15}.gds"
        row["gds_file"] = str(gds_path)

        c.write_gds(str(gds_path),with_metadata=False)
        abs_gds_path = Path.absolute(gds_path)
        imp1 = model/"geometries"/"Geometry 1"/"Import 1"
        imp1.property('filename',str(abs_gds_path))
        imp1.property("height",[5,0])
        imp1.property("importlayer",['on','off'])
        jmodel.geom("geom1").feature("imp1").importData()

        # --- 3) update selection box around moving end ---
        box1 = model / 'selections' / 'move'
        box1.property('xmax', xsize + 1)
        box1.property('xmin', xsize - 1)

        # --- 4) solve + evaluate stiffness ---
        model.solve()
        kxx, kyy, kzz = model.evaluate(['kxx', 'kyy', 'kzz'])[0]

        row["kxx"] = float(kxx)
        row["kyy"] = float(kyy)
        row["kzz"] = float(kzz)
        row["status"] = "ok"
        # model.save(str(mph_path))
    except Exception as e:
        row["error"] = f"{type(e).__name__}: {e}"

    return row
simulate_stiffness(100, 3, 1)

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm.notebook import tqdm
import time

# =========================
# User settings
# =========================
length_list = list(range(20, 151, 5))   # 20~100 step 5
width_list  = np.linspace(1, 5, 5).round(3).tolist()  # 1~4 共7个点
loops_list  = list(range(1, 3)) 
separation  = 15                                       # fixed for now

out_dir  = Path("gds_sweep")                           # folder for per-case gds files
out_dir.mkdir(parents=True, exist_ok=True)

csv_path = Path("spring_sweep_results.csv")

# selection box half-width in x (your +/-1 logic)
x_box_halfwidth = 1

# =========================
# Helper: append one row to CSV robustly
# =========================
def append_row_to_csv(row_dict, csv_path: Path):
    df_row = pd.DataFrame([row_dict])
    header = not csv_path.exists()
    df_row.to_csv(csv_path, mode="a", header=header, index=False)

# =========================
# Function to simulate stiffness of spring
# =========================
def simulate_stiffness(length, width, loop_num):
    row = dict(
        spring_length=float(length),
        spring_width=float(width),
        num_loops=int(loop_num),
        separation=float(separation),
        gds_file="",
        xsize=np.nan,
        kxx=np.nan, kyy=np.nan, kzz=np.nan,
        status="fail",
        error=""
    )

    try:
        # --- 1) build parametric layout ---
        c = spring_5um(
            spring_length=length,
            spring_width=width,
            separation=separation,
            num_loops=loop_num
        )
        c_wg = c.extract(['WG'])
        xsize = float(c_wg.xsize)
        row["xsize"] = xsize

        # --- 2) write per-case gds + import into COMSOL ---
        # Use a deterministic filename; keep some decimals safe for filenames.
        w_tag = f"{width}".replace(".", "p")
        gds_path = out_dir / f"spring_L{length}_W{w_tag}_N{loop_num}_S{separation}.gds"
        row["gds_file"] = str(gds_path)

        c.write_gds(str(gds_path),with_metadata=False)
        abs_gds_path = Path.absolute(gds_path)
        imp1 = model/"geometries"/"Geometry 1"/"Import 1"
        imp1.property('filename',str(abs_gds_path))
        imp1.property("height",[5,0])
        imp1.property("importlayer",['on','off'])
        jmodel.geom("geom1").feature("imp1").importData()

        # --- 3) update selection box around moving end ---
        box1 = model / 'selections' / 'move'
        box1.property('xmax', xsize + x_box_halfwidth)
        box1.property('xmin', xsize - x_box_halfwidth)

        # --- 4) solve + evaluate stiffness ---
        model.solve()
        kxx, kyy, kzz = model.evaluate(['kxx', 'kyy', 'kzz'])[0]

        row["kxx"] = float(kxx)
        row["kyy"] = float(kyy)
        row["kzz"] = float(kzz)
        row["status"] = "ok"
        mph_path = out_dir/f'spring_L{length}_W{w_tag}_N{loop_num}_S{separation}'
        # model.save(str(mph_path))
    except Exception as e:
        row["error"] = f"{type(e).__name__}: {e}"

    return row

# =========================
# Sweep
# =========================
cases = [(L, W, N) for L in length_list for W in width_list for N in loops_list]
t0 = time.time()

for spring_length, spring_width, num_loops in tqdm(cases, desc="COMSOL sweep", unit="case"):
    row = simulate_stiffness(spring_length, spring_width, num_loops)

    # --- 5) crash-safe append ---
    append_row_to_csv(row, csv_path)

t1 = time.time()
print(f"Done. CSV saved to: {csv_path.resolve()}")
print(f"GDS saved under: {out_dir.resolve()}")
print(f"Elapsed: {(t1 - t0):.1f} s")